In [2]:
import pandas as pd

In [3]:
import pymongo

client = pymongo.MongoClient('192.168.1.28', 27017)

client.admin.authenticate('admin', 'athena', mechanism = 'SCRAM-SHA-1')

db = client['Emails']

collection = db['godvinayak']

In [4]:
records = list(collection.find())

In [5]:
import re
import time
from dateutil.parser import parse
import datetime
from datetime import datetime
import base64
from bs4 import BeautifulSoup
class MailParser:
    def __init__(self):
        pass
    
    def parse_email(self,message):
        temp_dict = {}
        payld = message['payload'] # get payload of the message 
        headr = payld['headers'] # get header of the payload


        for one in headr: # getting the Subject
            if one['name'] == 'Subject':
                msg_subject = one['value']
                temp_dict['Subject'] = msg_subject
            elif one['name'] == 'Date':
                try:
                    msg_date = one['value']
                    date_parse = parse(msg_date)
                    m_date = date_parse.date()
                    temp_dict['Date'] = str(m_date)

                except:
                    pass
                #temp_dict['Date'] = str(m_date)
            elif one['name'] == 'From':
                msg_from = one['value']
                temp_dict['Sender'] = msg_from
            elif one['name'] == 'To':
                msg_from = one['value']
                temp_dict['Receiver'] = msg_from
            else:
                pass

        temp_dict['Snippet'] = message['snippet'] # fetching message snippet


        try:
            
            # Fetching message body
            mssg_parts = payld['parts'] # fetching the message parts
            part_one  = mssg_parts[0] # fetching first element of the part 
            part_body = part_one['body'] # fetching body of the message
            part_data = part_body['data'] # fetching data from the body
            clean_one = part_data.replace("-","+") # decoding from Base64 to UTF-8
            clean_one = clean_one.replace("_","/") # decoding from Base64 to UTF-8
            clean_two = base64.b64decode (bytes(clean_one, 'UTF-8')) # decoding from Base64 to UTF-8
            soup = BeautifulSoup(clean_two , "lxml" )
            mssg_body = soup.body()
            # mssg_body is a readible form of message body
            # depending on the end user's requirements, it can be further cleaned 
            # using regex, beautiful soup, or any other method
            temp_dict['Message_body'] = mssg_body

        except :
            pass
        return temp_dict

In [6]:
x=MailParser()

In [292]:
x.parse_email(records[0])

{'Date': '2019-05-27', 'Message_body': [<p>Swiss001 just uploaded a video
  Plane Has Birdstrike ON The Runway
  http://www.youtube.com/watch?v=PtPxcDMRv-8&amp;feature=em-uploademail
  You can unsubscribe from notifications for this user by visiting the  
  subscription center:
  http://www.youtube.com/subscription_manager
  </p>], 'Receiver': 'Vinayak Godse <godvinayak@gmail.com>', 'Sender': 'YouTube <noreply@youtube.com>', 'Snippet': 'Swiss001 has uploaded Plane Has Birdstrike ON The Runway Today&#39;s Video: Plane Has Bi... Youtube Logo 4:33 Plane Has Birdstrike ON The Runway Swiss001 You were sent this email because you chose to', 'Subject': 'Swiss001 just uploaded a video'}

In [8]:
len(records)

23769

In [9]:
any(x is None for x in records)

False

In [10]:
rec_email=[x.parse_email(record) for record in records]

/home/prakash/anaconda3/envs/tensorflow/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "http://intra.crysagi.com:8080/
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/prakash/anaconda3/envs/tensorflow/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "https://www.sciencedaily.com/releases/2017/04/170426131013.htm
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/prakash/anaconda3/envs/tensorflow/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "https://www.sciencedaily.com/releases/2018/12/181203131102.htm
" looks like a URL. Beautiful Soup is not an H

In [11]:
len(rec_email)

23769

### for getting group of threads

In [12]:
def get_threadid(records): #define over records
        return records['threadId'] #fetching threadId to list

In [13]:
threadid = [get_threadid(x) for x in records]

In [14]:
def creat_df(rec_email):
    df=pd.DataFrame(rec_email)
    df['threadId']=threadid
    df=df.dropna().reset_index()
    df=df.drop(['index','Snippet','Subject'],axis=1)
    df = df.fillna('')
    return df

In [15]:
df_email=creat_df(rec_email)

In [16]:
df_email.head()

,Date,Message_body,Receiver,Sender,threadId
0,2019-05-27,[<p>Swiss001 just uploaded a video Plane Has ...,Vinayak Godse <godvinayak@gmail.com>,YouTube <noreply@youtube.com>,16afaea769e0e1e6
1,2019-05-27,[<p>Vinayak's Digest Your Quora Digest ---...,godvinayak@gmail.com,Quora Digest <digest-noreply@quora.com>,16af99bc0fdfb81f
2,2019-05-27,[<p>Twitter &gt; https://twitter.com You'...,Vinayak Godse <godvinayak@gmail.com>,Twitter <notify@twitter.com>,16af96c4a9046f5d
3,2019-05-27,"[<p>Hello Vinayak, You've recently bought a T...",godvinayak@gmail.com,Lenskart <support@lenskart.com>,16af898a3651bb1f
4,2019-05-27,[<p>ÿ ICICI Bank You came. You transacted...,godvinayak@gmail.com,ICICI Bank Privilege Banking <marketing@custco...,16af86bb4eb8dddb


In [17]:
len(df_email)

16930

In [18]:
def get_string_msg(message):
        msg=[str(x) for x in message[0]]   # to get string type
        a=''.join(msg)
        return a

In [19]:
df_email['Message_body']=df_email.Message_body.apply(lambda x: get_string_msg(x))

In [20]:
# for splitting threads and remove replies
def get_main_email(x):
    x=x.split('wrote:') # splitting message from reply
    a=x[0].strip('[]')
    b=re.sub(r'^On.*\n?.*', '', a, flags=re.MULTILINE) # regex for removing extra line from email body
    return b

In [21]:
df_email['Message_body']=df_email.Message_body.apply(lambda x: get_main_email(x))

In [22]:
# assigning ids for each message body
def get_ids(x):
    return id(x)

In [23]:
df_email['ids'] = df_email.Message_body.apply(lambda x: get_ids(x)) # give unique id to each message

In [24]:
def unique_thread(df):
    unique_thread=pd.DataFrame(df.threadId.unique(),columns=['thread'])
    return unique_thread

In [25]:
unique_thread=unique_thread(df_email)

In [26]:
#grouping list of unique ids for threadId
def thread_id(x,df):
    a=df.ids[df.threadId == x] 
    return [x,a]

In [27]:
msr_thread=unique_thread.thread.apply(lambda x: thread_id(x,df_email))

In [296]:
msr_thread[0:5]

0    [16afaea769e0e1e6, [140079695702640]]
1         [16af99bc0fdfb81f, [1988135120]]
2         [16af96c4a9046f5d, [1987801568]]
3    [16af898a3651bb1f, [140079662125360]]
4         [16af86bb4eb8dddb, [1988153136]]
Name: thread, dtype: object

In [28]:
# creating dictionary such that keys are threadids and values are list of ids from dataframe
def creat_dict(msr_thread):
    dct_thread={}
    for i in range(len(msr_thread)):
        key=msr_thread[i][0]
        val=msr_thread[i][1].tolist()
        dct_thread[key]=val
    return dct_thread

In [29]:
dct_thread=creat_dict(msr_thread)

In [299]:
dct_thread['16a82d25f95e4733']

[140079661451208,
 140079662253232,
 1988020400,
 140079661561488,
 140079661381048,
 1992032288,
 140079676918560,
 1988005280,
 1988022560,
 1992274032]

In [30]:
df_email.to_csv('gv_email.csv',index=False)

### extracting signature

In [31]:
df_email.head()

,Date,Message_body,Receiver,Sender,threadId,ids
0,2019-05-27,Swiss001 just uploaded a video\r\nPlane Has Bi...,Vinayak Godse <godvinayak@gmail.com>,YouTube <noreply@youtube.com>,16afaea769e0e1e6,140079695702640
1,2019-05-27,Vinayak's Digest\r\nYour Quora Digest\r\n\r\n-...,godvinayak@gmail.com,Quora Digest <digest-noreply@quora.com>,16af99bc0fdfb81f,1988135120
2,2019-05-27,Twitter \r\n> https://twitter.com\r\n\r\nYou'v...,Vinayak Godse <godvinayak@gmail.com>,Twitter <notify@twitter.com>,16af96c4a9046f5d,1987801568
3,2019-05-27,"Hello Vinayak,\r\nYou've recently bought a Tor...",godvinayak@gmail.com,Lenskart <support@lenskart.com>,16af898a3651bb1f,140079662125360
4,2019-05-27,ÿ ICICI Bank \r\n\r\n You came. You transacted...,godvinayak@gmail.com,ICICI Bank Privilege Banking <marketing@custco...,16af86bb4eb8dddb,1988153136


In [216]:
from talon.signature.bruteforce import extract_signature
import re

def clean_data(message):
    message = message.replace('\r','').replace('\\r','').replace('>','').replace('<','').replace('//','').replace('\\n','\n').replace('Regards','\nRegards').replace('regards','\nregards')
    return message

def get_signature(email):
    return extract_signature(email)[1]

In [217]:
df_email['Message_body'] = df_email['Message_body'].apply(lambda x:clean_data(x))
df_email['Signature'] = df_email['Message_body'].apply(lambda x:get_signature(x))
df_email['Signature'] = df_email['Signature'].fillna('')
email_new = df_email.loc[df_email['Signature']!='']
email_new.head()

,Date,Message_body,Receiver,Sender,threadId,ids,Signature
144,2019-05-14,"Dear Sir,\n\nWe have received Rs.125,000/-.\n\...","Varad Godse <godvarad@gmail.com>, Finance <Fin...",Divya Agrawal <divya.agrawal@bennett.edu.in>,16ab45f77fa7d4e0,1988755056,Regards\nVarad Vinayak Godse\n/admissions/fina...
145,2019-05-14,Thank you very much Sir.\n\nMy father has tran...,Admissions Bennett University <admissions@benn...,Varad Godse <godvarad@gmail.com>,16a9b6a02ea160bc,140079661084976,Regards\nVarad Vinayak Godse
151,2019-05-13,Varad\nWe appreciate your concern\nI am changi...,Varad Godse <godvarad@gmail.com>,Admissions Bennett University <admissions@benn...,16a9b6a02ea160bc,1988013424,regards\nVarad Vinayak Godse
155,2019-05-13,"Dear Sir,\n\nSince I got the provisional admis...",Admissions Bennett University <admissions@benn...,Varad Godse <godvarad@gmail.com>,16a9b6a02ea160bc,1987999744,regards\nVarad Vinayak Godse
193,2019-05-09,Welcome on board Vinayak\nBesties\nRegistrar\n...,Varad Godse <godvarad@gmail.com>,Admissions Bennett University <admissions@benn...,16a9b6a02ea160bc,1988001072,Regards\n\nVinayak


In [218]:
len(email_new)

287

In [219]:
gv_email = email_new[df_email.Sender.str.lower().apply(lambda x: 'vinayak godse' in x or 'godvinayak' in x)].reset_index()
gv_email.head()

/home/prakash/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,index,Date,Message_body,Receiver,Sender,threadId,ids,Signature
0,469,2019-04-13,"Hi,\n\nI applied for domicile certificate for ...",edistrictgrievance@gmail.com,Vinayak Godse <godvinayak@gmail.com>,16a17658c5576835,140079662367712,regards\nVinayak
1,537,2019-04-07,"Yes, please send a person.\n\n\n\n\nRegards\nV...",Ankur Gupta6 <Ankur.Gupta6@aexp.com>,Vinayak Godse <godvinayak@gmail.com>,169ed0985f490cfa,140079662181992,Regards\nVinayak
2,602,2019-04-01,Hi thanks for the sending me offer of possessi...,SJP Infracon ltd <possession.sky@shrigroup.net>,vinayak godse <godvinayak@yahoo.com>,169d97c5af9306db,1988131008,regards\nVinayak Godse\n\nSent from my iPad\n\...
3,672,2019-03-26,"Ranjit,\n\nWe need it in a readable format. Ea...",Ranjit Yewale <ranjit@teckschool.com>,Vinayak Godse <godvinayak@gmail.com>,168cde88221e4bd0,140079662209760,regards\nVinayak
4,907,2019-03-05,You have to send it to the store in Dwarka 6 m...,Lenskart <support@lenskart.com>,Vinayak Godse <godvinayak@gmail.com>,1693a34dad89239e,140079661529968,Regards\nVinayak


In [220]:
def get_email_addr(s):
    if '<' in s:
        return s[s.find("<")+1:s.find(">")]
    else:
        return s

In [221]:
gv_email['Receiver'] = gv_email['Receiver'].fillna('')
gv_email['Receiver'] = gv_email['Receiver'].apply(lambda x:get_email_addr(x))

gv_email['Sender'] = gv_email['Sender'].fillna('')
gv_email['Sender'] = gv_email['Sender'].apply(lambda x:get_email_addr(x))

In [222]:
gv_email.head()

,index,Date,Message_body,Receiver,Sender,threadId,ids,Signature
0,469,2019-04-13,"Hi,\n\nI applied for domicile certificate for ...",edistrictgrievance@gmail.com,godvinayak@gmail.com,16a17658c5576835,140079662367712,regards\nVinayak
1,537,2019-04-07,"Yes, please send a person.\n\n\n\n\nRegards\nV...",Ankur.Gupta6@aexp.com,godvinayak@gmail.com,169ed0985f490cfa,140079662181992,Regards\nVinayak
2,602,2019-04-01,Hi thanks for the sending me offer of possessi...,possession.sky@shrigroup.net,godvinayak@yahoo.com,169d97c5af9306db,1988131008,regards\nVinayak Godse\n\nSent from my iPad\n\...
3,672,2019-03-26,"Ranjit,\n\nWe need it in a readable format. Ea...",ranjit@teckschool.com,godvinayak@gmail.com,168cde88221e4bd0,140079662209760,regards\nVinayak
4,907,2019-03-05,You have to send it to the store in Dwarka 6 m...,support@lenskart.com,godvinayak@gmail.com,1693a34dad89239e,140079661529968,Regards\nVinayak


In [223]:
gv_email['Receiver'] = gv_email['Receiver'].apply(lambda x:x.split(','))
gv_email['Sender'] = gv_email['Sender'].apply(lambda x:x.split(','))

In [224]:
import numpy as np
lst_col = 'Receiver'
test = pd.DataFrame({col:np.repeat(gv_email[col].values, gv_email[lst_col].str.len()) for col in gv_email.columns.difference([lst_col]) }).assign(**{lst_col:np.concatenate(gv_email[lst_col].values)})[gv_email.columns.tolist()]

In [225]:
lst_col = 'Sender'
test_df = pd.DataFrame({col:np.repeat(test[col].values, test[lst_col].str.len()) for col in test.columns.difference([lst_col]) }).assign(**{lst_col:np.concatenate(test[lst_col].values)})[test.columns.tolist()]

In [226]:
test_df['Receiver']=test_df.Receiver.apply(lambda x: x.strip())

In [227]:
test_df.head()

,index,Date,Message_body,Receiver,Sender,threadId,ids,Signature
0,469,2019-04-13,"Hi,\n\nI applied for domicile certificate for ...",edistrictgrievance@gmail.com,godvinayak@gmail.com,16a17658c5576835,140079662367712,regards\nVinayak
1,537,2019-04-07,"Yes, please send a person.\n\n\n\n\nRegards\nV...",Ankur.Gupta6@aexp.com,godvinayak@gmail.com,169ed0985f490cfa,140079662181992,Regards\nVinayak
2,602,2019-04-01,Hi thanks for the sending me offer of possessi...,possession.sky@shrigroup.net,godvinayak@yahoo.com,169d97c5af9306db,1988131008,regards\nVinayak Godse\n\nSent from my iPad\n\...
3,672,2019-03-26,"Ranjit,\n\nWe need it in a readable format. Ea...",ranjit@teckschool.com,godvinayak@gmail.com,168cde88221e4bd0,140079662209760,regards\nVinayak
4,907,2019-03-05,You have to send it to the store in Dwarka 6 m...,support@lenskart.com,godvinayak@gmail.com,1693a34dad89239e,140079661529968,Regards\nVinayak


### for recent email date and email_count

In [228]:
def per_count(df):
    person_count={}
    for value, count in df["Receiver"].value_counts().items():
        person_count[value]=count
    return person_count

In [229]:
dic=per_count(test_df)

In [230]:
def creat_count_df(dic):
    person_count=pd.DataFrame(list(dic.items()))
    person_count.columns=['receiver','count']
    dates=person_count.receiver.apply(lambda x: test_df.loc[test_df.Receiver == x]['Date'].tolist())
    person_count['dates'] = dates
    return person_count

In [231]:
person_count=creat_count_df(dic)

In [232]:
person_count.head()

,receiver,count,dates
0,shekhar.threepsolutions@gmail.com,1,[2017-12-17]
1,pranotikale2@gmail.com,1,[2018-03-31]
2,sachin.khalap@gmail.com,1,[2008-10-02]
3,anshu.malhotra@dsci.in,1,[2018-09-18]
4,dvivek1@gmail.com,2,"[2007-12-12, 2007-12-07]"


In [233]:
from datetime import datetime
def recent_email_date(dates):
    try:
        dates.sort(key = lambda date: datetime.strptime(str(date),'%Y-%m-%d')) # date format may change with data %d-day,%b-month,%Y-year
    except:
        pass
    return dates[-1]    

In [234]:
person_count['recent_date']=person_count.dates.apply(lambda x: recent_email_date(x))

In [235]:
person_count.head()

,receiver,count,dates,recent_date
0,shekhar.threepsolutions@gmail.com,1,[2017-12-17],2017-12-17
1,pranotikale2@gmail.com,1,[2018-03-31],2018-03-31
2,sachin.khalap@gmail.com,1,[2008-10-02],2008-10-02
3,anshu.malhotra@dsci.in,1,[2018-09-18],2018-09-18
4,dvivek1@gmail.com,2,"[2007-12-07, 2007-12-12]",2007-12-12


In [236]:
person_count=person_count.drop(['dates'],axis=1)
person_count.head(10)

,receiver,count,recent_date
0,shekhar.threepsolutions@gmail.com,1,2017-12-17
1,pranotikale2@gmail.com,1,2018-03-31
2,sachin.khalap@gmail.com,1,2008-10-02
3,anshu.malhotra@dsci.in,1,2018-09-18
4,dvivek1@gmail.com,2,2007-12-12
5,kailasbhat@yahoo.com,1,2018-12-27
6,h.kale0000@gmail.com,1,2016-08-20
7,animish.chavan7@gmail.com,1,2014-08-17
8,tarangshinde@gmail.com,2,2018-12-30
9,dhairyasheelb@gmail.com,1,2017-11-07


### for extracting info from signature

In [237]:
person_count['sender']=person_count.receiver.apply(lambda x: [ele for ele in test_df.loc[test_df['Receiver'] == x]['Sender'].unique()])

In [238]:
person_count.head()

,receiver,count,recent_date,sender
0,shekhar.threepsolutions@gmail.com,1,2017-12-17,[godvinayak@gmail.com]
1,pranotikale2@gmail.com,1,2018-03-31,[godvinayak@gmail.com]
2,sachin.khalap@gmail.com,1,2008-10-02,[godvinayak@gmail.com]
3,anshu.malhotra@dsci.in,1,2018-09-18,[godvinayak@gmail.com]
4,dvivek1@gmail.com,2,2007-12-12,[godvinayak@gmail.com]


In [239]:
person_count['sender']=person_count.sender.apply(lambda x: str(x).strip('[]').strip("''"))

In [240]:
person_count.head()

,receiver,count,recent_date,sender
0,shekhar.threepsolutions@gmail.com,1,2017-12-17,godvinayak@gmail.com
1,pranotikale2@gmail.com,1,2018-03-31,godvinayak@gmail.com
2,sachin.khalap@gmail.com,1,2008-10-02,godvinayak@gmail.com
3,anshu.malhotra@dsci.in,1,2018-09-18,godvinayak@gmail.com
4,dvivek1@gmail.com,2,2007-12-12,godvinayak@gmail.com


In [241]:
person_count['signature']=person_count.receiver.apply(lambda x: [ele for ele in test_df.loc[test_df['Receiver'] == x]['Signature'].unique()])

In [242]:
person_count['signature']=person_count.signature.apply(lambda x: str(x).strip('[]').strip("''"))

In [243]:
person_count.head()

,receiver,count,recent_date,sender,signature
0,shekhar.threepsolutions@gmail.com,1,2017-12-17,godvinayak@gmail.com,Regards\nVinayak
1,pranotikale2@gmail.com,1,2018-03-31,godvinayak@gmail.com,Regards\nVinayak
2,sachin.khalap@gmail.com,1,2008-10-02,godvinayak@gmail.com,regards
3,anshu.malhotra@dsci.in,1,2018-09-18,godvinayak@gmail.com,"Thanks,\n\nRudransh Mathur\nCo-Founder,\nNetwo..."
4,dvivek1@gmail.com,2,2007-12-12,godvinayak@gmail.com,"regards\n\nVinayak', 'regards\n\nV Godse"


In [244]:
person_count.signature[0]

'Regards\\nVinayak'

In [288]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

job_title=['CEO','C.T.O','Manager','chief','officer','supervisor','vice precident','Director','HR','founder','engineer','Developer','consultant']

def get_name(email,signature):
    person=re.findall(r'.*?\.*(.*)\@.' ,email)
    name=process.extractOne(str(person),signature.replace(',','\\n').split('\\n'))
    if name[1] > 80:
        return name[0]
    else:
        return ''
    
def get_company(email,signature):
    comp=re.findall(r'.*?\@(.*)\.' ,email)
    company=process.extractOne(str(comp),signature.replace(',','\\n').split('\\n'))
    if company[1] > 80:
        return company[0]
    else:
        return ''
    
def get_phone_no(signature):
    ph_no=re.findall(r'[\+\(]?[0-9][0-9 .\-\(\)]{8,}[0-9]',signature)
    return ",".join(ph_no)

def get_position(signature):
    positions = [process.extractOne(str(x),signature.replace(',','\\n').split('\\n')) for x in job_title]
    positions = [p for p in positions if p[1]>75]
    if len(positions) == 0:
        return ''
    else:
        return positions[0][0]

In [289]:
person_count['person_name'] = person_count.apply(lambda x: get_name(x.sender, x.signature), axis=1)
person_count['company_name'] = person_count.apply(lambda x: get_company(x.sender, x.signature), axis=1)
person_count['phone_no'] = person_count['signature'].apply(lambda x: get_phone_no(x))
person_count['position'] = person_count['signature'].apply(lambda x: get_position(x))

In [290]:
person_count.head()

,receiver,count,recent_date,sender,signature,person_name,company_name,phone_no,position
0,shekhar.threepsolutions@gmail.com,1,2017-12-17,godvinayak@gmail.com,Regards\nVinayak,Vinayak,,,
1,pranotikale2@gmail.com,1,2018-03-31,godvinayak@gmail.com,Regards\nVinayak,Vinayak,,,
2,sachin.khalap@gmail.com,1,2008-10-02,godvinayak@gmail.com,regards,,,,
3,anshu.malhotra@dsci.in,1,2018-09-18,godvinayak@gmail.com,"Thanks,\n\nRudransh Mathur\nCo-Founder,\nNetwo...",/godvinayak/rudransh,,+91 8007390859,Co-Founder
4,dvivek1@gmail.com,2,2007-12-12,godvinayak@gmail.com,"regards\n\nVinayak', 'regards\n\nV Godse",Vinayak',,,


In [291]:
person_count.to_csv('final_gv_email.csv',index=False)

### for all data

In [252]:
email_new.head()

,Date,Message_body,Receiver,Sender,threadId,ids,Signature
144,2019-05-14,"Dear Sir,\n\nWe have received Rs.125,000/-.\n\...","Varad Godse <godvarad@gmail.com>, Finance <Fin...",Divya Agrawal <divya.agrawal@bennett.edu.in>,16ab45f77fa7d4e0,1988755056,Regards\nVarad Vinayak Godse\n/admissions/fina...
145,2019-05-14,Thank you very much Sir.\n\nMy father has tran...,Admissions Bennett University <admissions@benn...,Varad Godse <godvarad@gmail.com>,16a9b6a02ea160bc,140079661084976,Regards\nVarad Vinayak Godse
151,2019-05-13,Varad\nWe appreciate your concern\nI am changi...,Varad Godse <godvarad@gmail.com>,Admissions Bennett University <admissions@benn...,16a9b6a02ea160bc,1988013424,regards\nVarad Vinayak Godse
155,2019-05-13,"Dear Sir,\n\nSince I got the provisional admis...",Admissions Bennett University <admissions@benn...,Varad Godse <godvarad@gmail.com>,16a9b6a02ea160bc,1987999744,regards\nVarad Vinayak Godse
193,2019-05-09,Welcome on board Vinayak\nBesties\nRegistrar\n...,Varad Godse <godvarad@gmail.com>,Admissions Bennett University <admissions@benn...,16a9b6a02ea160bc,1988001072,Regards\n\nVinayak


In [253]:
len(email_new)

287

In [260]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

job_title=['CEO','C.T.O','Manager','chief','officer','supervisor','vice precident','Director','HR','founder','engineer','Developer','consultant']

def get_name(email,signature):
    person=re.findall(r'.*?\.*(.*)\@.' ,email)
    name=process.extractOne(str(person),signature.split('\n'))
    if name[1] > 80:
        return name[0]
    else:
        return ''
    
def get_company(email,signature):
    comp=re.findall(r'.*?\@(.*)\.' ,email)
    company=process.extractOne(str(comp),signature.split('\n'))
    if company[1] > 80:
        return company[0]
    else:
        return ''
    
def get_phone_no(signature):
    ph_no=re.findall(r'[\+\(]?[0-9][0-9 .\-\(\)]{8,}[0-9]',signature)
    return ",".join(ph_no)

def get_position(signature):
    positions = [process.extractOne(str(x),signature.split('\n')) for x in job_title]
    positions = [p for p in positions if p[1]>75]
    if len(positions) == 0:
        return ''
    else:
        return positions[0][0]

In [261]:
email_new['person_name'] = email_new.apply(lambda x: get_name(x.Sender, x.Signature), axis=1)
email_new['company_name'] = email_new.apply(lambda x: get_company(x.Sender, x.Signature), axis=1)
email_new['phone_no'] = email_new['Signature'].apply(lambda x: get_phone_no(x))
email_new['position'] = email_new['Signature'].apply(lambda x: get_position(x))

/home/prakash/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/prakash/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/prakash/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [318]:
email_new[60:63]

,Date,Message_body,Receiver,Sender,threadId,ids,Signature,person_name,company_name,phone_no,position
2956,2018-09-18,ï¿½ \n\n http:\n\n--Click Here to unsubscribe ...,godvinayak@gmail.com,ZEE5 Support <norpely@zee5.com>,165ecb80d7f15fbe,140079662514704,--Click Here to unsubscribe from this newslett...,,,,
2967,2018-09-17,"Hi Vinayak, \nHereâ€™s a quick high level desi...",Vinayak Godse <godvinayak@gmail.com>,Rudransh Mathur <rudransh@noct.in>,165e70b3dd4a150b,2036631904,"Thanks,\n\nRudransh Mathur \nCo-Founder, \nNet...",Rudransh Mathur,www.noct.in,+91 8007390859,"Co-Founder,"
2973,2018-09-16,ï¿½ \n\n http:\n\n--Click Here to unsubscribe ...,godvinayak@gmail.com,ZEE5 Support <noreply@zee5.com>,165e35f021d16d01,140079662518064,--Click Here to unsubscribe from this newslett...,,,,
